In [14]:
import numpy as np
from scipy.stats import beta

# parameters and model set up

sigma = 5               # elasticity of substitution between critical goods
theta = 0.1             # between outside and critical good
kappa = 0.5             # fixed entry cost
z_h = {"A": 0.0, "B": 0.0}  # production subsidies for countries A and B
e_h = {"A": 0.0, "B": 0.0}  # entry subsidy for countries A and B
tau = 0.1                   # generic trade cost

imtax_Fij = {"A": {"A": 0.00, "B": 0.00, "C": 0.00}, # import tariff on critical goods from i to j
             "B": {"A": 0.00, "B": 0.00, "C": 0.00}}

extax_Fij = {"A": {"A": 0.00, "B": 0.00}, # export tax on critical goods from i to j
             "B": {"A": 0.00, "B": 0.00}}

imtax_Ihi = {"A": {"A": 0.00, "B": 0.00}, # import tariff on inputs from h to i
             "B": {"A": 0.00, "B": 0.00}}

extax_Ihi = {"A": {"A": 0.00, "B": 0.00}, # export tax on inputs from h to i
             "B": {"A": 0.00, "B": 0.00}}

alpha_c = {"A": 6, "B": 19}   # alpha for country risk, e.g. 6 (~86%), 9 (90%), 19 (95%), 99 (99%)
beta_c = {"A": 1, "B": 1}     # beta parameter (fix to 1)
alpha_i = 19                  # alpha parameter for idiosyncratic risk
beta_i = 1

cvar_alpha = 0.95  # confidence level for CVaR

# monte carlo setup
num_simulations = 25000
max_iterations = 500
min_iterations = 5 # min before checking convergence
iteration = 0
tolerance = 0.01  # relative error tolerance
z_alpha = 1.96    # critical z-value for 95% confidence
np.random.seed(42) # random number seed for reproducibility

In [16]:
# run model
# trade costs for critical goods from A to (A, B)
t_Fij = {
    i: {
        j: 1 if i == j else 1 + imtax_Fij.get(i, {}).get(j, 0) + extax_Fij.get(i, {}).get(j, 0) + tau
        for j in ("A", "B", "C") if j in imtax_Fij.get(i, {})
    }
    for i in ("A", "B", "C") if i in imtax_Fij
}

# trade costs for critical goods from A to (A, B)
t_Ihi = {
    h: {
        i: 1 if h == i else 1 + imtax_Ihi.get(h, {}).get(i, 0) + extax_Ihi.get(h, {}).get(i, 0) + tau
        for i in ("A", "B", "C") if i in imtax_Ihi.get(h, {})
    }
    for h in ("A", "B") if h in imtax_Ihi
}

# expected survival probability
expected_phi_h = {
    "A": (alpha_c["A"]/(alpha_c["A"] + beta_c["A"]))*(alpha_i/(alpha_i + beta_i)),
    "B": (alpha_c["B"]/(alpha_c["B"] + beta_c["B"]))*(alpha_i/(alpha_i + beta_i)),
}

# calculate input prices
p_Ih = {
    "A": (sigma / (sigma - 1)) * (1 / expected_phi_h["A"] - z_h["A"]),
    "B": (sigma / (sigma - 1)) * (1 / expected_phi_h["B"] - z_h["B"]),
}

# calculate price of final goods (p_Fi) = input price index (bar_P_Ii)
p_Fi = {i: np.sum([(p_Ih[h] * t_Ihi[h][i]) ** (1 - sigma) for h in ("A", "B")]) ** (1 / (1 - sigma)) for i in ("A", "B")}

# calculate price index for final goods consumed in country j
bar_P_Fj = {j: np.sum([(p_Fi[i] * t_Fij[i][j]) ** (1 - sigma) for i in ("A", "B")]) ** (1 / (1 - sigma)) for j in ("A", "B", "C")}

# expected demand for final goods
expected_x_Fij = {
    i: {j: (p_Fi[i] * t_Fij[i][j]) ** (-sigma) * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta)) for j in ("A", "B", "C")}
    for i in ("A", "B")
}

# expected demand for inputs
expected_X_Ihi = {
    h: {
        i: (p_Ih[h] * t_Ihi[h][i]) ** -sigma
        * sum(
            t_Fij[i][j] ** -sigma
            * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta))
            for j in ("A", "B", "C")
        )
        for i in ("A", "B")
    }
    for h in ("A", "B")
}

# n_h (number of entrants) for each h
n_h = {
    h: sum(expected_X_Ihi[h][i] for i in ("A", "B"))
    / ((sigma - 1) * (1 - e_h[h]) * kappa)
    * ((1 / expected_phi_h[h]) - z_h[h])
    for h in ("A", "B")
}

# expected survivors for each h
ns_h = {
    h: n_h[h] * expected_phi_h[h] for h in ("A", "B")
    for h in ("A", "B")
}

# initialise lists for storing results
results = []
cvar_results = []

# run the loop until convergence or max iterations
while iteration < max_iterations:
    iteration += 1   
    
    # generate risk factors
    D_h = {
        h: 1 - beta.rvs(alpha_c[h], beta_c[h], size = num_simulations)
        for h in ("A", "B")
    }
    fail_all_h = {
        h: np.random.rand(num_simulations) < D_h[h]
        for h in ("A", "B")
    }
    phi_h = {
        h: np.where(fail_all_h[h], 0, beta.rvs(alpha_i, beta_i, size = num_simulations))
        for h in ("A", "B")
    }

    # precompute this
    price_term = {
        h: 1 / expected_phi_h[h] - z_h[h] for h in ["A", "B"]
    }
    
    # compute rho_i directly for all samples
    rho_i = {
        i: (np.sum(
            [
                (
                    ((phi_h[h] / expected_phi_h[h]) ** ((sigma - 1) / sigma))
                    * (
                        price_term[h] ** (1 - sigma)
                    )
                    * (t_Ihi[h][i] ** (1 - sigma))
                )
                for h in ("A", "B")
            ],
            axis = 0,
        )
        / np.sum(
            [
                (
                    price_term[h] ** (1 - sigma)
                    * (t_Ihi[h][i] ** (1 - sigma))
                )
                for h in ["A", "B"]
            ],
            axis = 0,
        )) ** (sigma / (sigma - 1))
        for i in ["A", "B"]
    }

    # tariff revenue
    tariff_revenue = {
        j: sum(
            imtax_Fij.get(i, {}).get(j, 0) * rho_i.get(i, {}) * expected_x_Fij.get(i, {}).get(j, 0)
            for i in ("A", "B") if i != j
        ) + (sum(
            imtax_Ihi.get(i, {}).get(j, 0) * phi_h[i] / expected_phi_h[i] * expected_X_Ihi.get(i, {}).get(j, 0)
            for i in ("A", "B") if i != j
        ) if j in ("A", "B") else 0)
        + (sum(
            extax_Fij.get(j, {}).get(i, 0) * rho_i.get(j, {}) * expected_x_Fij.get(j, {}).get(i, 0)
            for i in ("A", "B") if j != i
        ) if j in ("A", "B") else 0)
        + (sum(
            extax_Ihi.get(j, {}).get(i, 0) * phi_h[j] / expected_phi_h[j] * expected_X_Ihi.get(j, {}).get(i, 0)
            for i in ("A", "B") if j != i
        ) if j in ("A", "B") else 0)
        for j in ("A", "B", "C")
    }
    
    # production/entry subsidy cost
    subsidy_cost = {
        h: phi_h[h] * n_h[h] * e_h[h] + z_h[h] * sum(
            phi_h[h] / expected_phi_h[h] * expected_X_Ihi[h][i] for i in ("A", "B")
        )
        for h in ("A", "B")
    }
    
    # outside good consumption
    x_0j = {
        j: 1
        + tariff_revenue.get(j, 0)
        - subsidy_cost.get(j, 0)
        - bar_P_Fj.get(j, 0) ** (-theta / (1 - theta))
        for j in ("A", "B", "C")
    }
    
    # compute utility U_j
    sum_term = {
        j: 
        (((rho_i["A"] * expected_x_Fij["A"][j]) ** ((sigma - 1) / sigma) + 
          (rho_i["B"] * expected_x_Fij["B"][j]) ** ((sigma - 1) / sigma)))
        for j in ("A", "B", "C")
    }
    
    U_j = {
        j: x_0j.get(j, 0) +
           (1 / theta) * (sum_term.get(j, 0) ** (theta * sigma / (sigma - 1)))
        for j in ("A", "B")
    }
    
    # Calculate VaR as a dictionary
    var_threshold = {
        j: np.percentile(U_j[j], (1 - cvar_alpha) * 100) for j in U_j.keys()
    }
    
    # Calculate CVaR as a dictionary
    cvar = {
        j: np.mean(U_j[j][U_j[j] <= var_threshold[j]]) for j in U_j.keys()
    }
    
    # append results
    cvar_results.append(cvar)
    results.append({j: np.mean(U_j[j]) for j in U_j.keys()})

    # extract utility and CVaR results for country "A"
    cvar_results_A = [cvar["A"] for cvar in cvar_results]
    results_A = [result["A"] for result in results]
    
    # calculate standard errors for country "A"
    cvar_sem_A = (
        np.std(cvar_results_A, ddof=1) / np.sqrt(len(cvar_results_A))
        if len(cvar_results_A) > 1
        else 0  # Use 0 if variance can't be computed
    )
    mean_sem_A = (
        np.std(results_A, ddof=1) / np.sqrt(len(results_A))
        if len(results_A) > 1
        else 0
    )
    
    # Calculate confidence interval widths for country "A"
    mean_ci_width_A = 2 * z_alpha * mean_sem_A
    cvar_ci_width_A = 2 * z_alpha * cvar_sem_A
    
    # Check convergence for country "A"
    mean_converged_A = (
        len(results_A) > min_iterations
        and (mean_ci_width_A / np.abs(np.mean(results_A))) < tolerance
    )
    cvar_converged_A = (
        len(cvar_results_A) > min_iterations
        and (cvar_ci_width_A / np.abs(np.mean(cvar_results_A))) < tolerance
    )
    
    if mean_converged_A and cvar_converged_A:
        print(f"Converged after {iteration} iterations.")
        break
else:
    print("Reached maximum iterations without convergence.")

# Extract utility and CVaR results for countries A and B
results_A = [result["A"] for result in results]
results_B = [result["B"] for result in results]
cvar_results_A = [cvar["A"] for cvar in cvar_results]
cvar_results_B = [cvar["B"] for cvar in cvar_results]

# Output final results
print(f"Overall mean utility (A): {np.mean(results_A):.3f}")
print(f"Overall mean utility (B): {np.mean(results_B):.3f}")
print(f"Mean CVaR (at α = {cvar_alpha}) (A): {np.mean(cvar_results_A):.3f}")
print(f"Mean CVaR (at α = {cvar_alpha}) (B): {np.mean(cvar_results_B):.3f}")

Converged after 28 iterations.
Overall mean utility (A): 9.775
Overall mean utility (B): 9.781
Mean CVaR (at α = 0.95) (A): 7.732
Mean CVaR (at α = 0.95) (B): 7.655


In [18]:
# multivariate (tariff, export tax, production sub) loop
# run the top two cells first to retrieve parameters

import itertools
import pandas as pd
from joblib import Parallel, delayed
import time

tariff_sequence = np.arange(0, 0.30, 0.05)
extax_sequence = np.arange(0, 0.35, 0.05)
prsub_sequence = np.arange(0, 0.35, 0.05)

# generate the parameter grid
parameter_grid = list(itertools.product(prsub_sequence,  # z_A
                                        prsub_sequence,  # z_B
                                        extax_sequence,  # extax_AB
                                        extax_sequence,  # extax_BA
                                        tariff_sequence, # tariff_AB
                                        tariff_sequence  # tariff_BA
                                       ))

# create a dataframe for the parameter grid
parameter_grid_df = pd.DataFrame(
    parameter_grid,
    columns = ["z_A", "z_B", "extax_AB", "extax_BA", "tariff_AB", "tariff_BA"]
)

# define logging function
def log_progress(current, total, interval = 500):
    if current % interval == 0 or current == total:
        print(f"Processed {current} rows out of {total} ({(current / total) * 100:.2f}%)...")

simulation_result = []

def run_simulation(row, index, total_rows):
    
    # extract parameters from the row
    z_A = row["z_A"]
    z_B = row["z_B"]
    extax_AB = row["extax_AB"]
    extax_BA = row["extax_BA"]
    tariff_AB = row["tariff_AB"]
    tariff_BA = row["tariff_BA"]

    no_convergence = 0

    # apply parameters to set tariff and subsidy rates
    imtax_Ihi = {"A": {"A": 0.00, "B": tariff_AB}, 
                 "B": {"A": tariff_BA, "B": 0.00}}
    extax_Ihi = {"A": {"A": 0.00, "B": extax_AB},
                 "B": {"A": extax_BA, "B": 0.00}}
    z_h = {"A": z_A, "B": z_B}

    # trade costs for critical goods from A to (A, B)
    t_Fij = {
        i: {
            j: 1 if i == j else 1 + imtax_Fij.get(i, {}).get(j, 0) + extax_Fij.get(i, {}).get(j, 0) + tau
            for j in ("A", "B", "C") if j in imtax_Fij.get(i, {})
        }
        for i in ("A", "B", "C") if i in imtax_Fij
    }
    
    # trade costs for critical goods from A to (A, B)
    t_Ihi = {
        h: {
            i: 1 if h == i else 1 + imtax_Ihi.get(h, {}).get(i, 0) + extax_Ihi.get(h, {}).get(i, 0) + tau
            for i in ("A", "B", "C") if i in imtax_Ihi.get(h, {})
        }
        for h in ("A", "B") if h in imtax_Ihi
    }
    
    # input price
    p_Ih = {
    "A": (sigma / (sigma - 1)) * (1 / expected_phi_h["A"] - z_h["A"]),
    "B": (sigma / (sigma - 1)) * (1 / expected_phi_h["B"] - z_h["B"]),
    }
    
    # calculate price of final goods (p_Fi) = input price index (bar_P_Ii)
    p_Fi = {i: np.sum([(p_Ih[h] * t_Ihi[h][i]) ** (1 - sigma) for h in ("A", "B")]) ** (1 / (1 - sigma)) for i in ("A", "B")}
    
    # calculate price index for final goods consumed in country j
    bar_P_Fj = {j: np.sum([(p_Fi[i] * t_Fij[i][j]) ** (1 - sigma) for i in ("A", "B")]) ** (1 / (1 - sigma)) for j in ("A", "B", "C")}
    
    # expected demand for final goods
    expected_x_Fij = {
        i: {j: (p_Fi[i] * t_Fij[i][j]) ** (-sigma) * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta)) for j in ("A", "B", "C")}
        for i in ("A", "B")
    }
    
    # expected demand for inputs
    expected_X_Ihi = {
        h: {
            i: (p_Ih[h] * t_Ihi[h][i]) ** -sigma
            * sum(
                t_Fij[i][j] ** -sigma
                * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta))
                for j in ("A", "B", "C")
            )
            for i in ("A", "B")
        }
        for h in ("A", "B")
    }
    
    # n_h (number of entrants) for each h
    n_h = {
        h: sum(expected_X_Ihi[h][i] for i in ("A", "B"))
        / ((sigma - 1) * (1 - e_h[h]) * kappa)
        * ((1 / expected_phi_h[h]) - z_h[h])
        for h in ("A", "B")
    }
    
    # expected survivors for each h
    ns_h = {
        h: n_h[h] * expected_phi_h[h] for h in ("A", "B")
        for h in ("A", "B")
    }
    
    # monte carlo setup
    iteration = 0
    utility_results = []
    cvar_results = []
    
    # run the loop until convergence or max iterations
    while iteration < max_iterations:
        iteration += 1   
        
        # generate risk factors
        D_h = {
            h: 1 - beta.rvs(alpha_c[h], beta_c[h], size = num_simulations)
            for h in ("A", "B")
        }
        fail_all_h = {
            h: np.random.rand(num_simulations) < D_h[h]
            for h in ("A", "B")
        }
        phi_h = {
            h: np.where(fail_all_h[h], 0, beta.rvs(alpha_i, beta_i, size = num_simulations))
            for h in ("A", "B")
        }
        
        # compute rho_i directly for all samples
        rho_i = {
            i: (np.sum(
                [
                    (
                        ((phi_h[h] / expected_phi_h[h]) ** ((sigma - 1) / sigma))
                        * (
                            price_term[h] ** (1 - sigma)
                        )
                        * (t_Ihi[h][i] ** (1 - sigma))
                    )
                    for h in ("A", "B")
                ],
                axis = 0,
            )
            / np.sum(
                [
                    (
                        price_term[h] ** (1 - sigma)
                        * (t_Ihi[h][i] ** (1 - sigma))
                    )
                    for h in ["A", "B"]
                ],
                axis = 0,
            )) ** (sigma / (sigma - 1))
            for i in ["A", "B"]
        }

        # tariff revenue
        tariff_revenue = {
            j: sum(
                imtax_Fij.get(i, {}).get(j, 0) * rho_i.get(i, {}) * expected_x_Fij.get(i, {}).get(j, 0)
                for i in ("A", "B") if i != j
            ) + (sum(
                imtax_Ihi.get(i, {}).get(j, 0) * phi_h[i] / expected_phi_h[i] * expected_X_Ihi.get(i, {}).get(j, 0)
                for i in ("A", "B") if i != j
            ) if j in ("A", "B") else 0)
            + (sum(
                extax_Fij.get(j, {}).get(i, 0) * rho_i.get(j, {}) * expected_x_Fij.get(j, {}).get(i, 0)
                for i in ("A", "B") if j != i
            ) if j in ("A", "B") else 0)
            + (sum(
                extax_Ihi.get(j, {}).get(i, 0) * phi_h[j] / expected_phi_h[j] * expected_X_Ihi.get(j, {}).get(i, 0)
                for i in ("A", "B") if j != i
            ) if j in ("A", "B") else 0)
            for j in ("A", "B", "C")
        }
        
        # production/entry subsidy cost
        subsidy_cost = {
            h: phi_h[h] * n_h[h] * e_h[h] + z_h[h] * sum(
                phi_h[h] / expected_phi_h[h] * expected_X_Ihi[h][i] for i in ("A", "B")
            )
            for h in ("A", "B")
        }
        
        # outside good consumption
        x_0j = {
            j: 1
            + tariff_revenue.get(j, 0)
            - subsidy_cost.get(j, 0)
            - bar_P_Fj.get(j, 0) ** (-theta / (1 - theta))
            for j in ("A", "B", "C")
        }
        
        # compute utility U_j
        sum_term = {
            j: 
            (((rho_i["A"] * expected_x_Fij["A"][j]) ** ((sigma - 1) / sigma) + 
              (rho_i["B"] * expected_x_Fij["B"][j]) ** ((sigma - 1) / sigma)))
            for j in ("A", "B", "C")
        }
        
        U_j = {
            j: x_0j.get(j, 0) +
               (1 / theta) * (sum_term.get(j, 0) ** (theta * sigma / (sigma - 1)))
            for j in ("A", "B")
        }
        
        # Calculate VaR as a dictionary
        var_threshold = {
            j: np.percentile(U_j[j], (1 - cvar_alpha) * 100) for j in U_j.keys()
        }
        
        # Calculate CVaR as a dictionary
        cvar = {
            j: np.mean([u for u in U_j[j] if u <= var_threshold[j]]) for j in U_j.keys()
        }
        
        # Append results
        cvar_results.append(cvar)
        utility_results.append({j: np.mean(U_j[j]) for j in U_j.keys()})
    
        # Extract utility and CVaR results for country "A"
        cvar_results_A = [cvar["A"] for cvar in cvar_results]
        utility_results_A = [utility["A"] for utility in utility_results]
        
        # calculate standard errors for country "A"
        cvar_sem_A = (
            np.std(cvar_results_A, ddof=1) / np.sqrt(len(cvar_results_A))
            if len(cvar_results_A) > 1
            else 0  # Use 0 if variance can't be computed
        )
        mean_sem_A = (
            np.std(results_A, ddof=1) / np.sqrt(len(results_A))
            if len(results_A) > 1
            else 0
        )
        
        # calculate confidence interval widths for country A
        mean_ci_width_A = 2 * z_alpha * mean_sem_A
        cvar_ci_width_A = 2 * z_alpha * cvar_sem_A
        
        # check convergence for country A
        mean_converged_A = (
            len(results_A) > min_iterations
            and (mean_ci_width_A / np.abs(np.mean(results_A))) < tolerance
        )
        cvar_converged_A = (
            len(cvar_results_A) > min_iterations
            and (cvar_ci_width_A / np.abs(np.mean(cvar_results_A))) < tolerance
        )
        
        if mean_converged_A and cvar_converged_A:
            # print(f"Converged after {iteration} iterations.")
            break
    else:
        print("Reached maximum iterations without convergence.")
        no_convergence = 1

    # after the monte carlo loop
    final_cvar = {k: np.mean([c[k] for c in cvar_results]) for k in cvar.keys()}
    final_utility = {k: np.mean([r[k] for r in utility_results]) for k in results[0].keys()}
    
    # save results
    simulation_result = {
        "tariff_AB": tariff_AB,
        "tariff_BA": tariff_BA,
        "extax_AB": extax_AB,
        "extax_BA": extax_BA,
        "z_A": z_A,
        "z_B": z_B,
        "utility_A": final_utility["A"],
        "utility_B": final_utility["B"],
        "cvar_A": final_cvar["A"],
        "cvar_B": final_cvar["B"],
        "symmetric": 1 if tariff_AB == tariff_BA else 0,
        "no_convergence": no_convergence
    }

    # log progress
    log_progress(index + 1, total_rows)
    
    return simulation_result

# run simulations in parallel with logging
start_time = time.time()
results = Parallel(n_jobs = -1, batch_size = 10)(  # batch_size = 10 fastest
    delayed(run_simulation)(
        {key: value for key, value in zip(parameter_grid_df.columns, row)},  # convert tuple to dict
        idx,
        len(parameter_grid_df)
    )
    for idx, row in enumerate(parameter_grid_df.itertuples(index=False))
)
end_time = time.time()

# convert the results to a dataframe
results_df = pd.DataFrame(results)

# save results
results_df = pd.DataFrame(results)
results_df.to_csv("output/multivariate_loop_asymmetric_risk.csv", index = False)

# final log
print(f"All rows processed. Total time: {end_time - start_time:.2f} seconds.")

Processed 500 rows out of 86436 (0.58%)...
Processed 1000 rows out of 86436 (1.16%)...
Processed 1500 rows out of 86436 (1.74%)...
Processed 2000 rows out of 86436 (2.31%)...
Processed 2500 rows out of 86436 (2.89%)...
Processed 3000 rows out of 86436 (3.47%)...
Processed 3500 rows out of 86436 (4.05%)...
Processed 4000 rows out of 86436 (4.63%)...
Processed 4500 rows out of 86436 (5.21%)...
Processed 5000 rows out of 86436 (5.78%)...
Processed 5500 rows out of 86436 (6.36%)...
Processed 6000 rows out of 86436 (6.94%)...
Processed 6500 rows out of 86436 (7.52%)...
Processed 7000 rows out of 86436 (8.10%)...
Processed 7500 rows out of 86436 (8.68%)...
Processed 8000 rows out of 86436 (9.26%)...
Processed 8500 rows out of 86436 (9.83%)...
Processed 9000 rows out of 86436 (10.41%)...
Processed 9500 rows out of 86436 (10.99%)...
Processed 10000 rows out of 86436 (11.57%)...
Processed 10500 rows out of 86436 (12.15%)...
Processed 11000 rows out of 86436 (12.73%)...
Processed 11500 rows out 

In [29]:
results_df

,tariff_AB,tariff_BA,extax_AB,extax_BA,z_A,z_B,utility_A,utility_B,cvar_A,cvar_B,symmetric,no_convergence
0,0.00,0.00,0.0,0.0,0.0,0.0,9.774758,9.780222,7.725090,7.648124,1,0
1,0.00,0.05,0.0,0.0,0.0,0.0,9.780089,9.771732,7.762139,7.672741,0,0
2,0.00,0.10,0.0,0.0,0.0,0.0,9.782759,9.766253,7.818286,7.722191,0,0
3,0.00,0.15,0.0,0.0,0.0,0.0,9.777806,9.757050,7.769598,7.676878,0,0
4,0.00,0.20,0.0,0.0,0.0,0.0,9.773654,9.751540,7.766180,7.691058,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
86431,0.25,0.05,0.3,0.3,0.3,0.3,9.765947,9.564595,7.686798,7.612739,0,0
86432,0.25,0.10,0.3,0.3,0.3,0.3,9.766660,9.556225,7.703922,7.628046,0,0
86433,0.25,0.15,0.3,0.3,0.3,0.3,9.762010,9.545238,7.678481,7.602207,0,0
86434,0.25,0.20,0.3,0.3,0.3,0.3,9.758579,9.537550,7.657421,7.580587,0,0
